In [1]:
# import tensorflow as tf
import numpy as np 
from sklearn.model_selection import train_test_split  


train_malicious = np.genfromtxt("./JSF20170324/malicious.vlog", delimiter=',')
train_normal    = np.genfromtxt("./JSF20170324/normal.vlog"   , delimiter=',')

X = np.concatenate((train_malicious, train_normal))
y = np.array([1]*len(train_malicious) + [0]*len(train_normal))
del(train_malicious)
del(train_normal)
y = np.reshape(y, [len(y), 1])
X = np.delete(X, np.s_[:1], axis=1)

print(X.shape)
print(y.shape)

(367028, 772)
(367028, 1)


In [2]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.05, random_state=831
)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)


(348676, 772)
(18352, 772)
(348676, 1)
(18352, 1)


In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
print(tf.__version__)

1.0.1


In [4]:
X_train = np.log(X_train+1)
X_valid = np.log(X_valid+1)

In [5]:
class Model:
    """>>> model = Model("Batch Norm", 32, 10)
    """
    def __init__(self, name, input_dim, output_dim, hidden_dims=[32, 32], use_batchnorm=True, activation_fn=tf.nn.relu, optimizer=tf.train.AdamOptimizer, lr=0.01):
        """ Constructor
        """
        with tf.variable_scope(name):
            # Placeholders are defined
            with tf.name_scope(name+'_input'):
                self.X = tf.placeholder(tf.float32, [None, input_dim], name='X')
                self.y = tf.placeholder(tf.float32, [None, output_dim], name='y')
                self.mode = tf.placeholder(tf.bool, name='train_mode')            
            
            # Loop over hidden layers
            net = self.X
            for i, h_dim in enumerate(hidden_dims):
                with tf.variable_scope('layer{}'.format(i)):
                    net = tf.layers.dense(net, h_dim)
                    
                    if use_batchnorm:
                        net = tf.layers.batch_normalization(net, training=self.mode)
                        
                    net = activation_fn(net)
            
            # Attach fully connected layers
            net = tf.contrib.layers.flatten(net)
            net = tf.layers.dense(net, output_dim)
            
            # self.loss = tf.nn.softmax_cross_entropy_with_logits(logits=net, labels=self.y)
            with tf.name_scope(name+'_loss'):
                self.loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=net, labels=self.y)
                # http://stackoverflow.com/questions/34240703/difference-between-tensorflow-tf-nn-softmax-and-tf-nn-softmax-cross-entropy-with
                self.loss = tf.reduce_mean(self.loss, name='loss')    

            with tf.name_scope(name+'_train'):
                # When using the batchnormalization layers,
                # it is necessary to manually add the update operations
                # because the moving averages are not included in the graph            
                update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=name)
                with tf.control_dependencies(update_ops):                     
                    self.train_op = optimizer(lr).minimize(self.loss)
            
            with tf.name_scope(name+'_Accuracy'):
                self.predicted = tf.cast(net > 0.5, dtype=tf.float32)
                self.accuracy = tf.reduce_mean(tf.cast(tf.equal(self.predicted, self.y), dtype=tf.float32))
            
            with tf.name_scope(name+'_ROC'):
                predi_bool = tf.cast(self.predicted, tf.bool)
                label_bool = tf.cast(self.y, tf.bool)
                # for TP
                TP_and = tf.logical_and(predi_bool, label_bool)
                # for TN
                tmp_or = tf.logical_or(predi_bool, label_bool)
                TN_not = tf.logical_not(tmp_or)
                # for FP
                label_not = tf.logical_not(label_bool)
                FP_and = tf.logical_and(predi_bool, label_not)
                # for FN
                predi_not = tf.logical_not(predi_bool)
                FN_and = tf.logical_and(predi_not, label_bool)
                # Add node
                self.TP = tf.reduce_sum(tf.cast(TP_and, tf.float32))
                self.TN = tf.reduce_sum(tf.cast(TN_not, tf.float32))
                self.FP = tf.reduce_sum(tf.cast(FP_and, tf.float32))
                self.FN = tf.reduce_sum(tf.cast(FN_and, tf.float32))
            
                # Precision Recall [ref](https://argcv.com/articles/1036.c)
                # wiki got better pic [wiki](https://en.wikipedia.org/wiki/Precision_and_recall)
                self.precision = self.TP / (self.TP + self.FP)
                self.recall    = self.TP / (self.TP + self.FN)
                
            # create a summary for our cost and accuracy 
            tf.summary.scalar("cost", self.loss)
            tf.summary.scalar("accuracy", self.accuracy)
            tf.summary.scalar("precision", self.precision)
            tf.summary.scalar("recall", self.recall)
            
            # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
            self.merged = tf.summary.merge_all()
            # self.merged = tf.summary.scalar("cost", self.loss)

In [6]:
class Solver:
    """Solver class
    
    This class will contain the model class and session
    
    Attributes
    ----------
    model : Model class
    sess : TF session
        
    Methods
    ----------
    train(X, y)
        Run the train_op and Returns the loss
        
    evalulate(X, y, batch_size=None)
        Returns "Loss" and "Accuracy"
        If batch_size is given, it's computed using batch_size
        because most GPU memories cannot handle the entire training data at once
            
    Example
    ----------
    >>> sess = tf.InteractiveSession()
    >>> model = Model("BatchNorm", 32, 10)
    >>> solver = Solver(sess, model)
    
    # Train
    >>> solver.train(X, y)
    
    # Evaluate
    >>> solver.evaluate(X, y)
    """
    def __init__(self, sess, model, writer):
        self.model = model
        self.sess = sess
        self.writer = writer
        
        self.writer.add_graph(sess.graph)
    def train(self, X, y):
        feed = {
            self.model.X: X,
            self.model.y: y,
            self.model.mode: True
        }
        train_op = self.model.train_op
        loss = self.model.loss
        
        return self.sess.run([train_op, loss], feed_dict=feed)
    
    def evaluate(self, X, y, batch_size=None):
        if batch_size:
            N = X.shape[0]
            
            total_loss = 0
            total_acc = 0
            total_TP = 0
            total_TN = 0
            total_FP = 0
            total_FN = 0
            for i in range(0, N, batch_size):
                
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                feed = {
                    self.model.X: X_batch,
                    self.model.y: y_batch,
                    self.model.mode: False
                }
                
                loss = self.model.loss
                accuracy = self.model.accuracy
                
                TP = self.model.TP
                TN = self.model.TN
                FP = self.model.FP
                FN = self.model.FN
                
                merged = self.model.merged
                
                summary, step_loss, step_acc, step_tp, step_tn, step_fp, step_fn = \
                    self.sess.run([merged, loss, accuracy, TP, TN, FP, FN], feed_dict=feed)
                
                total_loss += step_loss * X_batch.shape[0]
                total_acc += step_acc * X_batch.shape[0]
                total_TP += step_tp
                total_TN += step_tn
                total_FP += step_fp
                total_FN += step_fn
                
                self.writer.add_summary(summary, i)
            total_loss /= N
            total_acc /= N
            return (total_loss, total_acc, total_TP, total_TN, total_FP, total_FN)
            
        else:
            feed = {
                self.model.X: X,
                self.model.y: y,
                self.model.mode: False
            }
            
            loss = self.model.loss            
            accuracy = self.model.accuracy
            TP = self.model.TP
            TN = self.model.TN
            FP = self.model.FP
            FN = self.model.FN

            return self.sess.run([loss, accuracy, TP, TN, FP, FN], feed_dict=feed)

In [7]:
input_dim = 772
output_dim = 1
# N = 55000
N = len(X_train)

tf.reset_default_graph()
sess = tf.InteractiveSession()


# We create two models: one with the batch norm and other without
# def __init__(self, name, input_dim, output_dim, hidden_dims=[32, 32], use_batchnorm=True, activation_fn=tf.nn.relu, optimizer=tf.train.AdamOptimizer, lr=0.01):
bn = Model('batchnorm', input_dim, output_dim, hidden_dims=[1024, 1024, 512, 512, 256, 256, 64], use_batchnorm=True)
# nn = Model('no_norm', input_dim, output_dim, hidden_dims=[500, 250, 250, 100], use_batchnorm=False)
# bn = Model('batchnorm', input_dim, output_dim, hidden_dims=[2,2], use_batchnorm=True)
#nn = Model('no_norm', input_dim, output_dim, hidden_dims=[2,2], use_batchnorm=False)

# tensorboard setting 
logs_path = "./file_writer/"
# tensorboard --logdir=path/to/log-directory
bn_file_writer = tf.summary.FileWriter(logs_path+"bnlog/")
#nn_file_writer = tf.summary.FileWriter(logs_path+"nn/")

# We create two solvers: to train both models at the same time for comparison
# Usually we only need one solver class
bn_solver = Solver(sess, bn, bn_file_writer)
#nn_solver = Solver(sess, nn, nn_file_writer)

In [8]:
epoch_n = 300
##batch_size = 32
batch_size = 500

# Save Losses and Accuracies every epoch
# We are going to plot them later
train_losses = []
train_accs = []
train_ROC = []

valid_losses = []
valid_accs = []
valid_ROC = []

In [9]:
print("Number of batch : ", N//batch_size)
print("Total traning data: ", N)
print("Batch size :", batch_size)

def get_minibatches_index(n, batch_size, shuffle=False):
    """
    Used to shjuffle the dataset at each iteration.
    """

    idx_list = np.arange(n, dtype="int32")

    if shuffle:
        np.random.shuffle(idx_list)

    minibatches = []
    minibatch_start = 0
    for i in range(n // batch_size):
        minibatches.append(idx_list[minibatch_start:
                                        minibatch_start + batch_size])
        minibatch_start += batch_size

    if (minibatch_start != n):
        # Make a minibatch out of what is left
        minibatches.append(idx_list[minibatch_start:])

    return minibatches

minibatches = get_minibatches_index(len(X_train), batch_size, shuffle=True)


Number of batch :  697
Total traning data:  348676
Batch size : 500


In [10]:
print(X_train.shape)
print(y_train.shape)

(348676, 772)
(348676, 1)


In [11]:
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(epoch_n):
    ''' ##not use batch
    for _ in range(N//batch_size):
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        
        _, bn_loss = bn_solver.train(X_batch, y_batch)
        _, nn_loss = nn_solver.train(X_batch, y_batch)       
    '''

    for inds in minibatches[:-1]:
        # print("\rbatch %d / %d"%(b_ind, n_batches), end="")
        # sys.stdout.flush()
        X_batch = X_train[inds, :]
        y_batch = y_train[inds]
        _, bn_loss = bn_solver.train(X_batch, y_batch)
        #_, nn_loss = nn_solver.train(X_batch, y_batch)
    
    
    ##b_loss, b_acc = bn_solver.evaluate(mnist.train.images, mnist.train.labels, batch_size)
    b_loss, b_acc, b_tp, b_tn, b_fp, b_fn = bn_solver.evaluate(X_train, y_train, batch_size)
    ##n_loss, n_acc = nn_solver.evaluate(mnist.train.images, mnist.train.labels, batch_size)
    #n_loss, n_acc, n_tp, n_tn, n_fp, n_fn = nn_solver.evaluate(X_train, y_train, batch_size)
    
    # Save train losses/acc
    '''
    train_losses.append([b_loss, n_loss])
    train_accs.append([b_acc, n_acc])
    train_ROC.append([[b_tp, b_tn, b_fp, b_fn],[n_tp, n_tn, n_fp, n_fn]])
    # print(f'[Epoch {epoch}-TRAIN] Batchnorm Loss(Acc): {b_loss:.5f}({b_acc:.2%}) vs No Batchnorm Loss(Acc): {n_loss:.5f}({n_acc:.2%})')
    print('[Epoch %d-TRAIN] Batchnorm Loss(Acc): %.5f(%.2f) vs No Batchnorm Loss(Acc): %.5f(%.2f)' % (epoch, b_loss, b_acc, n_loss, n_acc))
    print('Batchnorm Loss(Acc): %.5f(%.2f), TP: %d, TN: %d, FP, %d, FN: %d' % (b_loss, b_acc,b_tp, b_tn, b_fp, b_fn))
    print('No Batchnorm Loss(Acc): %.5f(%.2f), TP: %d, TN: %d, FP, %d, FN: %d' % (n_loss, n_acc, n_tp, n_tn, n_fp, n_fn))
    '''
    train_losses.append(b_loss)
    train_accs.append(b_acc)
    train_ROC.append([b_tp, b_tn, b_fp, b_fn])
    # print(f'[Epoch {epoch}-TRAIN] Batchnorm Loss(Acc): {b_loss:.5f}({b_acc:.2%}) vs No Batchnorm Loss(Acc): {n_loss:.5f}({n_acc:.2%})')
    print('[Epoch %d-TRAIN] Batchnorm Loss(Acc): %.5f(%.2f)' % (epoch, b_loss, b_acc))
    print('Batchnorm Loss(Acc): %.5f(%.2f), TP: %d, TN: %d, FP, %d, FN: %d' % (b_loss, b_acc,b_tp, b_tn, b_fp, b_fn))
    # print('No Batchnorm Loss(Acc): %.5f(%.2f), TP: %d, TN: %d, FP, %d, FN: %d' % (n_loss, n_acc, n_tp, n_tn, n_fp, n_fn))
    
    ##b_loss, b_acc = bn_solver.evaluate(mnist.validation.images, mnist.validation.labels)
    b_loss, b_acc, b_tp, b_tn, b_fp, b_fn = bn_solver.evaluate(X_valid, y_valid)
    ##n_loss, n_acc = nn_solver.evaluate(mnist.validation.images, mnist.validation.labels)
    #n_loss, n_acc, n_tp, n_tn, n_fp, n_fn = nn_solver.evaluate(X_valid, y_valid)
    
    # Save valid losses/acc
    '''
    valid_losses.append([b_loss, n_loss])
    valid_accs.append([b_acc, n_acc])
    valid_ROC.append([[b_tp, b_tn, b_fp, b_fn],[n_tp, n_tn, n_fp, n_fn]])
    print(f'[Epoch {epoch}-VALID] Batchnorm Loss(Acc): {b_loss:.5f}({b_acc:.2%}) vs No Batchnorm Loss(Acc): {n_loss:.5f}({n_acc:.2%})')
    print('[Epoch %d-VALID] Batchnorm Loss(Acc): %.5f(%.2f) vs No Batchnorm Loss(Acc): %.5f(%.2f)' % (epoch, b_loss, b_acc, n_loss, n_acc))
    print('Batchnorm Loss(Acc): %.5f(%.2f), TP: %d, TN: %d, FP, %d, FN: %d' % (b_loss, b_acc,b_tp, b_tn, b_fp, b_fn))
    print('No Batchnorm Loss(Acc): %.5f(%.2f), TP: %d, TN: %d, FP, %d, FN: %d' % (n_loss, n_acc, n_tp, n_tn, n_fp, n_fn))
    '''
    valid_losses.append(b_loss)
    valid_accs.append(b_acc)
    valid_ROC.append([b_tp, b_tn, b_fp, b_fn])
    # print(f'[Epoch {epoch}-TRAIN] Batchnorm Loss(Acc): {b_loss:.5f}({b_acc:.2%}) vs No Batchnorm Loss(Acc): {n_loss:.5f}({n_acc:.2%})')
    print('[Epoch %d-VALID] Batchnorm Loss(Acc): %.5f(%.2f)' % (epoch, b_loss, b_acc))
    print('Batchnorm Loss(Acc): %.5f(%.2f), TP: %d, TN: %d, FP, %d, FN: %d' % (b_loss, b_acc,b_tp, b_tn, b_fp, b_fn))
    # print('No Batchnorm Loss(Acc): %.5f(%.2f), TP: %d, TN: %d, FP, %d, FN: %d' % (n_loss, n_acc, n_tp, n_tn, n_fp, n_fn))
    
    print()

[Epoch 0-TRAIN] Batchnorm Loss(Acc): 0.04096(0.99)
Batchnorm Loss(Acc): 0.04096(0.99), TP: 131557, TN: 213166, FP, 543, FN: 3410
[Epoch 0-VALID] Batchnorm Loss(Acc): 0.04420(0.99)
Batchnorm Loss(Acc): 0.04420(0.99), TP: 6987, TN: 11149, FP, 34, FN: 182

[Epoch 1-TRAIN] Batchnorm Loss(Acc): 0.08117(0.98)
Batchnorm Loss(Acc): 0.08117(0.98), TP: 128802, TN: 213310, FP, 399, FN: 6165
[Epoch 1-VALID] Batchnorm Loss(Acc): 0.08113(0.98)
Batchnorm Loss(Acc): 0.08113(0.98), TP: 6845, TN: 11158, FP, 25, FN: 324

[Epoch 2-TRAIN] Batchnorm Loss(Acc): 0.03391(0.99)
Batchnorm Loss(Acc): 0.03391(0.99), TP: 132272, TN: 213027, FP, 682, FN: 2695
[Epoch 2-VALID] Batchnorm Loss(Acc): 0.03832(0.99)
Batchnorm Loss(Acc): 0.03832(0.99), TP: 7021, TN: 11146, FP, 37, FN: 148

[Epoch 3-TRAIN] Batchnorm Loss(Acc): 0.13859(0.96)
Batchnorm Loss(Acc): 0.13859(0.96), TP: 120453, TN: 213518, FP, 191, FN: 14514
[Epoch 3-VALID] Batchnorm Loss(Acc): 0.13995(0.96)
Batchnorm Loss(Acc): 0.13995(0.96), TP: 6408, TN: 11173, 

KeyboardInterrupt: 

In [12]:
# load test

test_malicious = np.genfromtxt("./JSF20170324/test/malicious.vlog", delimiter=',')
test_normal    = np.genfromtxt("./JSF20170324/test/normal.vlog"   , delimiter=',')

X_test = np.concatenate((test_malicious, test_normal))
y_test = np.array([1]*len(test_malicious) + [0]*len(test_normal))
del(test_malicious)
del(test_normal)
y_test = np.reshape(y_test, [len(y_test), 1])
X_test = np.delete(X_test, np.s_[:1], axis=1)
X_test = np.log(X_test+1)
print(X_test.shape)
print(y_test.shape)

(206762, 772)
(206762, 1)


In [15]:
b_loss, b_acc, b_tp, b_tn, b_fp, b_fn = bn_solver.evaluate(X_test, y_test, batch_size)

In [16]:
print(b_loss, b_acc, b_tp, b_tn, b_fp, b_fn)

0.312768378784 0.963465295507 86636.0 112572.0 594.0 6960.0


In [ ]:
nn_solver.evaluate(X_test, y_test)

In [ ]:
def plot_compare(loss_list: list, ylim=None, title=None) -> None:
    
    bn = [i[0] for i in loss_list]
    nn = [i[1] for i in loss_list]
    
    plt.figure(figsize=(15, 10))
    plt.plot(bn, label='With BN')
    plt.plot(nn, label='Without BN')
    if ylim:
        plt.ylim(ylim)
        
    if title:
        plt.title(title)
    plt.legend()
    plt.grid('on')
    plt.show()

In [ ]:
plot_compare(train_losses, title='Training Loss at Epoch')

In [ ]:
plot_compare(train_accs, [0, 1.0], title="Training Acc at Epoch")

In [ ]:
plot_compare(valid_losses, title='Validation Loss at Epoch')

In [ ]:
plot_compare(valid_accs, [0, 1.], title='Validation Acc at Epoch')